# Questão 1

In [12]:
import numpy as np
from numpy import genfromtxt

In [13]:
kc2 = genfromtxt('kc2.csv', delimiter=',')
kc2.shape

(214, 22)

In [14]:
kc2_X = kc2[:, :-1] # (214,21)
kc2_y = kc2[:, [-1]] # (214,1) classe 0 ou 1. 
kc2_X.shape, kc2_y.shape


((214, 21), (214, 1))

## item a) 

#### K-fold

In [95]:
def compute_metrics_binary(y_pred, y_val):
    y_val = y_val.ravel()
    accuracy = np.mean(y_pred == y_val)

    tp = np.sum((y_val == 1) & (y_pred == 1))
    fp = np.sum((y_val != 1) & (y_pred == 1))
    fn = np.sum((y_val == 1) & (y_pred != 1))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        
    return accuracy, recall, precision, f1 

In [ ]:
def compute_accuracies(y_pred, y_val):
    by_class_accuracies = []
    accuracies = []

    if  y_val.shape[1] > 1: # dados one hot encoded
         y_val = np.argmax(y_val, axis=1)    

    y_pred = y_pred.ravel()
    y_val = y_val.ravel().astype(int)  

    classes = np.unique(y_val)
 
    y_val = y_val.ravel()

    for k in classes: 
        y_pred_k = y_pred[y_val == k]
        y_val_k = y_val[y_val == k]
        class_acc = np.mean(y_pred_k==y_val_k)
        by_class_accuracies.append(class_acc)
            
    accuracy = np.mean(y_pred == y_val)
    accuracies.append(accuracy)

    return accuracies, by_class_accuracies

In [120]:
# 10 fold cross validation:
def k_fold_cross_validation(X, y, predicao, k=10):
    
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    fold_size = len(X) // k

    recalls = []
    precisions = []
    f1s = []
    accuracies = []

    for i in range(k):

        start = i * fold_size
        end = (i + 1) * fold_size if i != k - 1 else len(X)  
        val_indices = indices[start:end]
        train_indices = np.concatenate((indices[:start], indices[end:]))

        X_treino, y_treino = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        mean = X_treino.mean(axis=0)
        std = X_treino.std(axis=0, ddof=1)

        std[std == 0] = 1
        X_treino_norm = (X_treino - mean) / std
        X_val_norm = (X_val - mean) / std

        y_pred = predicao(X_treino_norm, y_treino, X_val_norm)
        
        accuracies_fold, recalls_fold, precisions_fold, f1s_fold = compute_metrics_binary(y_pred, y_val)

        accuracies.append(accuracies_fold)
        recalls.append(recalls_fold)
        precisions.append(precisions_fold)
        f1s.append(f1s_fold)


    return np.array(accuracies), np.array(recalls), np.array(precisions), np.array(f1s)




#### KNN

In [1]:
def distancia_euclidiana(X_treino,X_teste):
    return np.sqrt(np.sum((X_teste[:, np.newaxis, :] - X_treino[np.newaxis, :, :]) ** 2, axis=2))

def distancia_mahalonobis(X_treino,X_teste):
    cov = np.cov(X_treino, rowvar=False)
    cov_inv = np.linalg.inv(cov)
    distancias = []
    for x in X_teste:
        diff = X_treino - x
        d = np.sqrt(np.sum((diff @ cov_inv) * diff, axis=1))
        distancias.append(d)
    return np.array(distancias)


In [ ]:
# 1 = euclidiana, 2 = mahalonobis 
def predicao_KNN(X_treino, y_treino, X_teste, k = 1, distancia_tipo = 1):

    if distancia_tipo == 1:
        distancias = distancia_euclidiana(X_treino,X_teste)    
    elif distancia_tipo == 2:
        distancias = distancia_mahalonobis(X_treino,X_teste)
    else: 
        print('erro, escolha 1 para distancia euclidiana ou 2 para mahalonobis')
    predicoes = []

    for i in range(X_teste.shape[0]): # para cada entrada
        idx_k_menores = np.argsort(distancias[i])[:k]
        classes = y_treino[idx_k_menores] 
        valores, contagens = np.unique(classes, return_counts=True)
        predicoes.append(valores[np.argmax(contagens)])
    return np.array(predicoes)


#### Árvore de decisão

In [3]:
from sklearn.tree import DecisionTreeClassifier

def predicaoArvore(X_treino, y_treino, X_val):
    clf = DecisionTreeClassifier(random_state=0)
    clf.fit(X_treino, y_treino)
    return clf.predict(X_val)

## item b)

In [127]:
acc, recall, precision, f1 = k_fold_cross_validation(kc2_X, kc2_y, predicaoArvore, k=10)
print(f'Acurácia média para árvore de decisão: {acc.mean()}\nDesvio padrão para para acurácia: {acc.std()}\n' ) 
print(f'Revocação média para árvore de decisão:  {recall.mean()}\nDesvio padrão para para revocação: {recall.std()}\n')
print(f'Precisão média para árvore de decisão:  {precision.mean()}\nDesvio padrão para para precisão: {precision.std()}\n')
print(f'F1-score média para árvore de decisão:  {f1.mean()}\nDesvio padrão para para F1-score: {f1.std()}\n')

Acurácia média para árvore de decisão: 0.733904761904762
Desvio padrão para para acurácia: 0.08821351133905984

Revocação média para árvore de decisão:  0.7167782217782218
Desvio padrão para para revocação: 0.14266368500090312

Precisão média para árvore de decisão:  0.7654428904428905
Desvio padrão para para precisão: 0.1537095610953507

F1-score média para árvore de decisão:  0.7231645962732919
Desvio padrão para para F1-score: 0.09661160397931445



K = 1

Distância Euclidiana

In [132]:
#k = 1, distancia euclidiana
acc, recall, precision, f1 = k_fold_cross_validation(kc2_X, kc2_y, predicao_KNN)
print(f'Acurácia média para KNN: {acc.mean()}\nDesvio padrão para para acurácia: {acc.std()}\n' ) 
print(f'Revocação média para KNN:  {recall.mean()}\nDesvio padrão para para revocação: {recall.std()}\n')
print(f'Precisão média para KNN:  {precision.mean()}\nDesvio padrão para para precisão: {precision.std()}\n')
print(f'F1-score média para KNN:  {f1.mean()}\nDesvio padrão para para F1-score: {f1.std()}\n')

Acurácia média para KNN: 0.7815238095238095
Desvio padrão para para acurácia: 0.08528591369522831

Revocação média para KNN:  0.7785173160173161
Desvio padrão para para revocação: 0.13121098262976832

Precisão média para KNN:  0.7883119658119659
Desvio padrão para para precisão: 0.14643709900328208

F1-score média para KNN:  0.7711825972978423
Desvio padrão para para F1-score: 0.10382633978271592



K = 1

Distância de Mahalonobis

In [ ]:
#k = 1, mahalonobis 
precisao = lambda Xt, yt, Xv: predicao_KNN(Xt, yt, Xv, k=1, distancia_tipo=2)
acc, recall, precision, f1 = k_fold_cross_validation(kc2_X, kc2_y, predicao_KNN)
print(f'Acurácia média para KNN: {acc.mean()}\nDesvio padrão para para acurácia: {acc.std()}\n' ) 
print(f'Revocação média para KNN:  {recall.mean()}\nDesvio padrão para para revocação: {recall.std()}\n')
print(f'Precisão média para KNN:  {precision.mean()}\nDesvio padrão para para precisão: {precision.std()}\n')
print(f'F1-score média para KNN:  {f1.mean()}\nDesvio padrão para para F1-score: {f1.std()}\n')


Acurácia média para KNN: 0.7649523809523809
Desvio padrão para para acurácia: 0.11823401983655979

Revocação média para KNN:  0.7658599733599734
Desvio padrão para para revocação: 0.16517265937157785

Precisão média para KNN:  0.7623776223776224
Desvio padrão para para precisão: 0.14181747876100237

F1-score média para KNN:  0.7541750530171581
Desvio padrão para para F1-score: 0.13990713918189415



K = 5

Distância Euclidiana

In [ ]:
precisao = lambda Xt, yt, Xv: predicao_KNN(Xt, yt, Xv, k=5, distancia_tipo=1)
acc, recall, precision, f1 = k_fold_cross_validation(kc2_X, kc2_y, predicao_KNN)
print(f'Acurácia média para KNN: {acc.mean()}\nDesvio padrão para para acurácia: {acc.std()}\n' ) 
print(f'Revocação média para KNN:  {recall.mean()}\nDesvio padrão para para revocação: {recall.std()}\n')
print(f'Precisão média para KNN:  {precision.mean()}\nDesvio padrão para para precisão: {precision.std()}\n')
print(f'F1-score média para KNN:  {f1.mean()}\nDesvio padrão para para F1-score: {f1.std()}\n')

Acurácia média para KNN: 0.7878095238095237
Desvio padrão para para acurácia: 0.11024014191250339

Revocação média para KNN:  0.8105014430014432
Desvio padrão para para revocação: 0.13449350990152212

Precisão média para KNN:  0.7829814629814631
Desvio padrão para para precisão: 0.09886436858458449

F1-score média para KNN:  0.7905216868477739
Desvio padrão para para F1-score: 0.09800880348270978



K = 5

Distância de Mahalonobis

In [140]:
precisao = lambda Xt, yt, Xv: predicao_KNN(Xt, yt, Xv, k=5, distancia_tipo=2)
acc, recall, precision, f1 = k_fold_cross_validation(kc2_X, kc2_y, predicao_KNN)
print(f'Acurácia média para KNN: {acc.mean()}\nDesvio padrão para para acurácia: {acc.std()}\n' ) 
print(f'Revocação média para KNN:  {recall.mean()}\nDesvio padrão para para revocação: {recall.std()}\n')
print(f'Precisão média para KNN:  {precision.mean()}\nDesvio padrão para para precisão: {precision.std()}\n')
print(f'F1-score média para KNN:  {f1.mean()}\nDesvio padrão para para F1-score: {f1.std()}\n')

Acurácia média para KNN: 0.7752380952380952
Desvio padrão para para acurácia: 0.08761904761904762

Revocação média para KNN:  0.7611499611499613
Desvio padrão para para revocação: 0.11319620290381605

Precisão média para KNN:  0.7928282828282829
Desvio padrão para para precisão: 0.12455835662729857

F1-score média para KNN:  0.7709915764378008
Desvio padrão para para F1-score: 0.09382073558781652

